### tester différentes méthodes de detection de pic sur toute les masses, pour pouvoir tester DBSCAN et comparer les méthodes

In [ ]:
from skimage.feature import peak_local_max
from sklearn.cluster import DBSCAN
import skimage
import baseline_correction
from read_chroma import read_chromato_and_chromato_cube
import numpy as np

In [ ]:
file_path = '/home/camille/Documents/app/data/input/A-F-028-817822-droite-ReCIVA.cdf'
# estimation with filtering_factor_estimation.ipynb
mod_time = 1.7

In [ ]:
chromato_tic_preprocessed, time_rn, chromato_cube_preprocessed, sigma, mass_range = read_chromato_and_chromato_cube(file_path, mod_time, pre_process=True)

In [ ]:
filtering_factor = 50 #TODO
sigma = skimage.restoration.estimate_sigma(chromato_tic_preprocessed, channel_axis=None)
dynamic_threshold_fact = filtering_factor * sigma * 100 / np.max(chromato_tic_preprocessed)
dynamic_threshold_fact

In [ ]:



coordinates_all_mass = []
for i in range(len(chromato_cube_preprocessed)):
    coordPLM=peak_local_max(chromato_cube_preprocessed[i], min_distance=1, threshold_rel=0.2)
    coordPERS= pers_hom_kernel(i,m_chromato=chromato_cube_preprocessed[i], seuil=None,threshold_abs=0.2)
    for x,y in coord:
        coordinates_all_mass.append([i,x,y])
## save output
coordinates_all_mass = np.delete(coordinates_all_mass, 0, -1) # supp l'info de la mass
len(coordinates_all_mass)
clustering = DBSCAN(eps=0.5, min_samples=5).fit(coordinates_all_mass) # lance DBscan
clusters = []
for i in range((np.max(clustering.labels_) + 1)):
    clusters.append([])for i, (t1, t2) in enumerate(coordinates_all_mass):
    clusters[clustering.labels_[i]].append([t1, t2])# clusters = np.array(clusters)
# Coordinates of the point with the biggest intensity in the cluster for every clusters
coordinates = []
for cluster in clusters:
    if (len(cluster) > 1):
        #coord = cluster[np.argmax(np.array([chromato[coord[0], coord[1]] for coord in cluster]))]
        arr=np.array(cluster)
        median_x = np.median(arr[:, 0])  # Median of first coordinate
        median_y = np.median(arr[:, 1])  # Median of second coordinate
        coord=[round(median_x),round(median_y)]
    else:
        coord = cluster[0]
    coordinates.append(coord)
coordinates = np.array(coordinates)